In [2]:
import pandas as pd

df=pd.read_csv('magaza_yorumlari_duygu_analizi.csv', encoding='utf-16')

df

,Görüş,Durum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz
1,hizli teslimat tesekkürler,Tarafsız
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz
...,...,...
11424,biraz fazla ısınıyor ama sorumsuz çalışıyor,Tarafsız
11425,orijinal ürün fakat ambalaj bize ulaşana kadar...,Tarafsız
11426,bu cihazı 1 yıldır kullanıyorum cihazı çok beğ...,Olumsuz
11427,Bir sene olmadı ayar düğmesine basıyorum ekran...,Tarafsız


In [3]:
#burada manuel olarak label encoding yaptım
custom_mapping = {'Olumlu': 2, 'Tarafsız': 1, 'Olumsuz': 0}
def custom_transform(x):
    return custom_mapping[x]

# fit_transform yerine transform ve custom_transform kullanma
durum_encoded = df["Durum"].map(custom_transform)

df["Durum_encoded"]=durum_encoded
df

,Görüş,Durum,Durum_encoded
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz,0
1,hizli teslimat tesekkürler,Tarafsız,1
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu,2
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz,0
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz,0
...,...,...,...
11424,biraz fazla ısınıyor ama sorumsuz çalışıyor,Tarafsız,1
11425,orijinal ürün fakat ambalaj bize ulaşana kadar...,Tarafsız,1
11426,bu cihazı 1 yıldır kullanıyorum cihazı çok beğ...,Olumsuz,0
11427,Bir sene olmadı ayar düğmesine basıyorum ekran...,Tarafsız,1


In [4]:
import re # noktalama işaretlerini kaldırmak için türkçe için türkçe karakterleri ekledim

yorum = re.sub('[^a-zA-ZğüşıöçĞÜŞİÖÇ ]', ' ', df['Görüş'][0])
yorum=yorum.lower()
yorum=yorum.split()

yorum

['ses',
 'kalitesi',
 've',
 'ergonomisi',
 'rezalet',
 'sony',
 'olduğu',
 'için',
 'aldım',
 'ama',
 'de',
 'fiyatına',
 'çin',
 'replika',
 'ürün',
 'alsaydım',
 'çok',
 'çok',
 'daha',
 'iyiydi',
 'kesinlikle',
 'tavsiye',
 'etmiyorum']

In [5]:
import nltk

from nltk.stem.porter import PorterStemmer  #kelimeleri gövde şeklinde alıyorum 
from nltk.corpus import stopwords  # durma sözcüklerini atacağım
nltk.download('punkt')
nltk.download('stopwords')


ps=PorterStemmer()


butun_yorumlar = []

for x in range(11429):
    yorum = re.sub('[^a-zA-ZğüşıöçĞÜŞİÖÇ ]', ' ', str(df['Görüş'][x]))
    yorum = yorum.lower()
    yorum = yorum.split()

    yorumm = []
    for kelime in yorum:
        if not kelime in set(stopwords.words('turkish')):
            yorumm.append(ps.stem(kelime))

    butun_yorumlar.append(" ".join(yorumm))





[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
butun_yorumlar

['se kalitesi ergonomisi rezalet soni olduğu aldım fiyatına çin replika ürün alsaydım iyiydi kesinlikl tavsiy etmiyorum',
 'hizli teslimat tesekkürl',
 'se olayı süper gece çalıştır sıkıntı yok kablo uzun işinizi rahat ettirir çekm olayı son derec güzel içiniz rahat olsun diğerlerini saymıyorum bile',
 'geldi bigün kullandık hemen bozoldu hiçtavsiy etmem',
 'kulaklığın sesi kalit falan değil aleti öve öve bitiremeyen yorumlar şüpheli tizler yok olan boğuk çıkıyor ba kaba saba ben buradayım diyor kalit yok i̇yi se arayanlara tavsiy etmem hayatımda aldığım ilk snopi marka üründü onu yorumlara güvenerek aldım pişman oldum hepsiburada nın saht yorumlara karşı önlem alması gerekiyor artık',
 'giriş seviyesindeki kullanıcılar kabul edilebilir sonuçlar veren bir len foku motoru makro çekimleri kalit sonuçlar veriyor şunu belirtmek lazım belli bir süre kullandıktan sonra kalit bir len ihtiyacı hissedeceksiniz',
 'kullanisli baya',
 'dezavantajlar pahalı ürün br merhabalar len başka yerlerd tl 

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

cv = CountVectorizer(max_features=1000)
X = cv.fit_transform(butun_yorumlar).toarray()
y = df.iloc[:, 2].values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

sv = CatBoostClassifier()
sv.fit(x_train, y_train)
y_pred = sv.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy



Learning rate set to 0.088581
0:	learn: 1.0813296	total: 12.3ms	remaining: 12.3s
1:	learn: 1.0689667	total: 25.5ms	remaining: 12.7s
2:	learn: 1.0579980	total: 38.8ms	remaining: 12.9s
3:	learn: 1.0468765	total: 49.2ms	remaining: 12.3s
4:	learn: 1.0365993	total: 61.6ms	remaining: 12.3s
5:	learn: 1.0278137	total: 74.4ms	remaining: 12.3s
6:	learn: 1.0206177	total: 86.3ms	remaining: 12.2s
7:	learn: 1.0133495	total: 97.8ms	remaining: 12.1s
8:	learn: 1.0053806	total: 111ms	remaining: 12.2s
9:	learn: 0.9983082	total: 126ms	remaining: 12.4s
10:	learn: 0.9922096	total: 140ms	remaining: 12.6s
11:	learn: 0.9871821	total: 153ms	remaining: 12.6s
12:	learn: 0.9816554	total: 166ms	remaining: 12.6s
13:	learn: 0.9765584	total: 180ms	remaining: 12.7s
14:	learn: 0.9727912	total: 194ms	remaining: 12.7s
15:	learn: 0.9693696	total: 207ms	remaining: 12.7s
16:	learn: 0.9653674	total: 224ms	remaining: 13s
17:	learn: 0.9623104	total: 241ms	remaining: 13.2s
18:	learn: 0.9584579	total: 260ms	remaining: 13.4s
19:	l

0.6933508311461067